In [ ]:
import re,os,operator
import pandas as pd

def findAllFiles(path, extension):
    filenames = os.listdir(path)
    return [ os.path.join(path,filename) for filename in filenames if filename.endswith( extension ) ]

def findAllFileNames():
    cur_path = os.getcwd()
    relative_path = 'data/raw/reddit/daywise'
    path = os.path.join(cur_path,relative_path)
    extension =".csv"
    file_lists = findAllFiles(path,extension)
    return file_lists

def parseCandidates(str):
    if not str: return []
    tickers_2 = re.findall('^([A-Z]{2})$',str)
    tickers_3 = re.findall('^([A-Z]{3})$',str)
    tickers_4 = re.findall('^([A-Z]{4})$',str)
    return tickers_2+tickers_3+tickers_4

def getCandidatesFreq(df):
    tmp1 = list(df['title'].fillna('').apply(parseCandidates))
    tmp2 = list(df['selftext'].fillna('').apply(parseCandidates))
    df_list = pd.DataFrame({'title_list':tmp1,'selftext_list':tmp2})
    df_list['comb_list'] = df_list['title_list']+df_list['selftext_list']
    word_count = {}
    for i in range(len(df_list)):
        tmp = df_list['comb_list'].iloc[i]
        tmp = list(dict.fromkeys(tmp))
        for j in tmp:
            if j not in word_count:
                word_count[j] = 0
            word_count[j] += 1
    return word_count

def getCandidates(df,col_name):
    tmp = df[col_name].fillna('').apply(parseCandidates)
    tmp = [i for i in tmp if len(i) > 0]
    stock_list = []
    for i in tmp:
        stock_list.extend(i)
    stock_list = list(dict.fromkeys(stock_list))
    return stock_list

def getAllCandidates(filename):
    df = pd.read_csv(filename)
    stock_list_title = getCandidates(df,'title')
    stock_list_selftext = getCandidates(df,'selftext')
    stock_list = list(dict.fromkeys(stock_list_title+stock_list_selftext))
    return stock_list

In [ ]:
file_list = findAllFileNames()
complete_candidates_count = {}
for i in file_list:
    df = pd.read_csv(i)
    candidates_count = getCandidatesFreq(df)
    for candidate,count in candidates_count.items():
        if candidate in complete_candidates_count.keys():
            complete_candidates_count[candidate] += count
        else:
            complete_candidates_count[candidate] = count
#complete_candidates_count = dict( sorted(complete_candidates_count.items(), key=operator.itemgetter(1),reverse=True))
complete_candidates_count

In [ ]:
df = pd.DataFrame.from_dict(complete_candidates_count,orient='index').reset_index()
df.columns =['possible_ticker','count']
# saving the dataframe 
df.to_csv('data/clean/reddit/possible_tickers_count.csv') 

In [ ]:
file_list = findAllFileNames()
complete_ticker_list = []
for i in file_list:
    stock_list = getAllCandidates(i)
    complete_ticker_list.append(stock_list)
complete_ticker_list

In [ ]:
all_possible_tickers = []
for i in complete_ticker_list:
    all_possible_tickers.extend(i)
all_possible_tickers = list(dict.fromkeys(all_possible_tickers))
all_possible_tickers

In [ ]:
dict = {'possible_tickers': all_possible_tickers}  
df = pd.DataFrame(dict)
# saving the dataframe 
df.to_csv('possible_tickers.csv') 